In [ ]:
# HIDDEN CELL
import sys, os

# Importing argopy in dev mode:
on_rtd = os.environ.get('READTHEDOCS', None) == 'True'
if not on_rtd:
    sys.path.insert(0, "/Users/gmaze/git/github/euroargodev/argopy")
    import git
    import argopy
    from argopy.options import OPTIONS
    # argopy.set_options(local_ftp='/Volumes/Data/ARGO')
    # argopy.set_options(mode='expert')
    print("argopy:", argopy.__version__, 
          "\nsrc:", argopy.__file__, 
          "\nbranch:", git.Repo(search_parent_directories=True).active_branch.name, 
          "\noptions:", OPTIONS)

import xarray as xr
xr.set_options(display_style="html");
xr.set_options(display_style="text");

# Fetching Argo data

To access Argo data, you need to use a data fetcher. You can import and instantiate the default argopy data fetcher
like this:

In [ ]:
from argopy import DataFetcher as ArgoDataFetcher
argo_loader = ArgoDataFetcher()

Then, you can request data for a specific **space/time domain**, for a given **float** or for a given vertical **profile**.

## For a space/time domain

In [ ]:
ds = argo_loader.region([-75, -45, 20, 30, 0, 10, '2011-01-01', '2011-06']).to_xarray()
ds

Note that:

- the constraints on time is not mandatory: if not specified, the fetcher will return all data available in this region.
- the last time bound is exclusive: that's why here we specifiy June to retrieve data collected in May.

## For one or more floats

In [ ]:
ds = argo_loader.float(6902746).to_xarray()
ds

To fetch data for a collection of floats, input them in a list:

In [ ]:
ds = argo_loader.float([6902746, 6902755]).to_xarray()
ds

## For one or more profiles

In [ ]:
ds = argo_loader.profile(6902755, 12).to_xarray()
ds

To fetch data for more than one profile, input them in a list:

In [ ]:
ds = argo_loader.profile(6902755, [3, 12]).to_xarray()
ds

# Fetchers features

## Data sources

We have implemented two Argo data fetchers:

- ``errdap``, to retrieve data online directly from the Ifremer erddap server,
- ``localftp``, to retrieve data from your local copy of the GDAC ftp.

You have several ways to specify which fetcher type you want to use:

- using **argopy** global options:

In [ ]:
import argopy
argopy.set_options(src='erddap')

- in a temporary context:

In [ ]:
with argopy.set_options(src='erddap'):
    ds = ArgoDataFetcher().profile(6902746, 34).to_xarray()

- when instantiating the data fetcher:

In [ ]:
ds = ArgoDataFetcher(src='erddap').profile(6902746, 34).to_xarray()

Data fetching with the ``localftp`` data source will require you to specify the path toward your local copy of the GDAC ftp server.

## User levels: standard vs expert

Because:  
- Argo measurements come from many different models of floats or sensors,
- the data management workflow is distributed between more than 10 Data Assembly Centers all around the world, 
- the quality control of in situ measurements of autonomous platforms is realy a matter of ocean and data experts,  
the Argo data set is very complex, with thousands of different variables, tens of reference tables and a user manual more than 150 pages.

In order to ease Argo data analysis for the vast majority of *standard* users, we implemented in **argopy** different levels of verbosity and data processing to hide or simply remove variables only meaningful to *experts*.

If you don't know in which category you would place yourself, try to answer the following questions:

- what is a WMO number ?
- what is the difference between Delayed and Real Time data mode ?
- what is an adjusted parameter ?
- what a QC flag of 3 means ?

If you answered to no more than 1 question, you probably would feel more confortable within the *standard* user mode.

By default, all **argopy** data fetchers are set to work with a *standard* user mode, the other possible mode is *expert*.

In *standard* mode, fetched data are automatically filtered to account for their quality (only good are retained) and level of processing by the data centers (wether they looked at the data briefly or not).

If you want to change the user mode, or simply makes it explicit, you can use:

- **argopy** global options:

In [ ]:
import argopy
argopy.set_options(mode='standard')

- a temporary context:

In [ ]:
with argopy.set_options(mode='standard'):
    ds = ArgoDataFetcher().profile(6902746, 34).to_xarray()

- option when instantiating the data fetcher:

In [ ]:
ds = ArgoDataFetcher(mode='standard').profile(6902746, 34).to_xarray()

If we compare data fetched for one profile with each modes, we have:

In [ ]:
with argopy.set_options(mode='standard'):
    ds = ArgoDataFetcher().profile(6902746, 34).to_xarray()
    print(ds.data_vars)

In [ ]:
with argopy.set_options(mode='expert'):
    ds = ArgoDataFetcher().profile(6902746, 34).to_xarray()
    print(ds.data_vars)

# Manipulating data

By default fetched data are returned in memory as [xarray.DataSet](http://xarray.pydata.org/en/stable/data-structures.html#dataset). 
From there, it is easy to convert data to other formats like a [Pandas dataframe](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe):

In [ ]:
ds = ArgoDataFetcher().profile(6902746, 34).to_xarray()
df = ds.to_dataframe()
df

or to export it to files:

In [ ]:
ds = ArgoDataFetcher().region([-85,-45,10.,20.,0,100., '2011-01-01', '2011-05']).to_xarray()
ds.to_netcdf('my_selection.nc')

# Fetching Argo meta-data

Since the Argo measurements data set is quite complex, it comes with a collection of index, or lookup tables with meta data. These index help you determine what you can expect before retrieving the full set of measurements.

In [ ]:
from argopy import IndexFetcher as ArgoIndexFetcher
index_loader = ArgoIndexFetcher()

In [ ]:
idx = index_loader.region([-75, -45, 20, 30, '2011-01-01', '2011-05'])
idx

In [ ]:
idx.to_dataframe()

# Performance

## Caching data

- in a temporary context:

In [ ]:
with argopy.set_options(cachedir='mycache_folder'):
    ds = ArgoDataFetcher(cache=True).profile(6902746, 34).to_xarray()

- when instantiating the data fetcher:

In [ ]:
ds = ArgoDataFetcher(cache=True, cachedir='mycache_folder').profile(6902746, 34).to_xarray()